In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import json
from datetime import datetime
import logging
from pathlib import Path 
import concurrent.futures
import tqdm
import yfinance as yf
import pandas as pd
import os
from os.path import join
from datetime import datetime
from pathlib import Path 

In [2]:
# Read the csv from stagging
path = '../../data/intermediate_02/financial-sheets.csv'
ticker_file_all = pd.read_csv(path)
ticker = 'ITRN'

In [3]:
ticker_file = ticker_file_all[ticker_file_all['ticker'] == ticker]

In [5]:
# Get the company data from yfinance
stock_info = yf.Ticker(ticker)
hist = stock_info.history(period="max", interval = "1mo")

# Transform data to add growth info
hist = hist.reset_index()
hist['Date'] = pd.to_datetime(hist['Date'])
hist['year'] = hist['Date'].dt.year
hist.dropna(inplace=True)

In [6]:
hist

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,year
0,2005-09-01,-0.160403,-0.168209,-0.158573,-0.160403,4087200.0,0.0,0,2005
1,2005-10-01,-0.160403,-0.161379,-0.138081,-0.158573,2031300.0,0.0,0,2005
2,2005-11-01,-0.158573,-0.187848,-0.154304,-0.177358,2896000.0,0.0,0,2005
3,2005-12-01,-0.175528,-0.208950,-0.173211,-0.194557,2252900.0,0.0,0,2005
4,2006-01-01,-0.199924,-0.227979,-0.190288,-0.216635,2063600.0,0.0,0,2006
...,...,...,...,...,...,...,...,...,...
260,2023-03-01,22.459999,23.059999,20.879999,21.780001,492000.0,0.0,0,2023
261,2023-04-01,21.920000,22.250000,20.740000,20.799999,357300.0,0.0,0,2023
262,2023-05-01,20.920000,24.000000,20.350000,22.840000,746200.0,0.0,0,2023
263,2023-06-01,22.730000,25.500000,22.389999,23.990000,382200.0,0.0,0,2023


In [7]:
# Convert 'Date' column in 'hist' dataframe to string format
hist['Date'] = hist['Date'].astype(str)
# Extract the 'YYYY-MM' format from 'Date' column in 'ticker_file' dataframe
ticker_file['YearMonth'] = ticker_file['Date'].str.slice(0, 7)
# Filter 'hist' dataframe based on 'YYYY-MM' values from 'ticker_file'
filtered_hist = hist[hist['Date'].str.slice(0, 7).isin(ticker_file['YearMonth'])]

C:\Users\germa\AppData\Local\Temp/ipykernel_22276/3425731416.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ticker_file['YearMonth'] = ticker_file['Date'].str.slice(0, 7)


In [9]:
ticker_file['YearMonth']

26251    2009-12
26252    2010-12
26253    2011-12
26254    2012-12
26255    2013-12
26256    2014-12
26257    2015-12
26258    2016-12
26259    2017-12
26260    2018-12
26261    2019-12
26262    2020-12
26263    2021-12
26264    2022-12
Name: YearMonth, dtype: object

In [10]:
filtered_hist

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,year
58,2009-12-01,-0.411037,-0.433872,-0.401839,-0.406597,291100.0,0.0,0,2009
71,2010-12-01,-0.553083,-0.619565,-0.540205,-0.608775,376800.0,0.0,0,2010
84,2011-12-01,-0.470012,-0.518941,-0.452961,-0.506338,189600.0,0.0,0,2011
101,2012-12-01,-0.618219,-0.641851,-0.602622,-0.641379,382700.0,0.0,0,2012
119,2013-12-01,-1.109750,-1.218942,-1.087466,-1.205572,619500.0,0.0,0,2013
138,2014-12-01,17.369110,18.862709,17.007756,17.698345,369400.0,0.0,0,2014
154,2015-12-01,16.820898,17.179500,14.811064,15.786793,310200.0,0.0,0,2015
170,2016-12-01,21.696809,23.258978,21.323623,22.998617,477700.0,0.0,0,2016
186,2017-12-01,32.040356,32.202358,30.060337,30.780344,788300.0,0.0,0,2017
202,2018-12-01,30.840987,32.582680,27.218628,29.701473,688900.0,0.0,0,2018


In [11]:
# Format to datetime
filtered_hist['Date'] = pd.to_datetime(filtered_hist['Date'])
hist['Date'] = pd.to_datetime(hist['Date'])
# Find the earliest date in the filtered_hist dataframe
earliest_date = filtered_hist['Date'].min()
# Filter 'hist' dataframe for rows with a date greater than the earliest date
filtered_hist_after_earliest = hist[hist['Date'].dt.year > earliest_date.year]
# Find the earliest date in the filtered_hist dataframe
earliest_date = filtered_hist['Date'].min()
# Filter 'hist' dataframe for rows with a date greater than the earliest date
filtered_hist_after_earliest = hist[hist['Date'].dt.year < earliest_date.year]

C:\Users\germa\AppData\Local\Temp/ipykernel_22276/2001008434.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_hist['Date'] = pd.to_datetime(filtered_hist['Date'])


In [12]:
filtered_hist_after_earliest

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,year
0,2005-09-01,-0.160403,-0.168209,-0.158573,-0.160403,4087200.0,0.0,0,2005
1,2005-10-01,-0.160403,-0.161379,-0.138081,-0.158573,2031300.0,0.0,0,2005
2,2005-11-01,-0.158573,-0.187848,-0.154304,-0.177358,2896000.0,0.0,0,2005
3,2005-12-01,-0.175528,-0.208950,-0.173211,-0.194557,2252900.0,0.0,0,2005
4,2006-01-01,-0.199924,-0.227979,-0.190288,-0.216635,2063600.0,0.0,0,2006
5,2006-02-01,-0.215537,-0.222002,-0.180895,-0.207365,1831000.0,0.0,0,2006
6,2006-03-01,-0.207365,-0.212610,-0.183823,-0.184555,1455700.0,0.0,0,2006
8,2006-04-01,-0.188791,-0.214177,-0.188791,-0.205058,599100.0,0.0,0,2006
9,2006-05-01,-0.204565,-0.214054,-0.168951,-0.183246,2365400.0,0.0,0,2006
10,2006-06-01,-0.182753,-0.191009,-0.168335,-0.172648,2380600.0,0.0,0,2006


In [13]:
# Filter the resulting dataframe for rows with the same month as the earliest date
appended_hist = filtered_hist_after_earliest[filtered_hist_after_earliest['Date'].dt.month == earliest_date.month]

In [14]:
appended_hist

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,year
3,2005-12-01,-0.175528,-0.208950,-0.173211,-0.194557,2252900.0,0.0,0,2005
16,2006-12-01,-0.196678,-0.206167,-0.179426,-0.185464,912300.0,0.0,0,2006
30,2007-12-01,-0.136807,-0.148906,-0.132287,-0.147178,859900.0,0.0,0,2007
44,2008-12-01,-0.216545,-0.250766,-0.200557,-0.205045,1223100.0,0.0,0,2008


In [15]:
# Concatenate the filtered_hist dataframe with the appended_hist dataframe
prices_filtered = pd.concat([filtered_hist, appended_hist]).sort_values(by='year').reset_index(drop=True)

In [16]:
prices_filtered

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,year
0,2005-12-01,-0.175528,-0.208950,-0.173211,-0.194557,2252900.0,0.0,0,2005
1,2006-12-01,-0.196678,-0.206167,-0.179426,-0.185464,912300.0,0.0,0,2006
2,2007-12-01,-0.136807,-0.148906,-0.132287,-0.147178,859900.0,0.0,0,2007
3,2008-12-01,-0.216545,-0.250766,-0.200557,-0.205045,1223100.0,0.0,0,2008
4,2009-12-01,-0.411037,-0.433872,-0.401839,-0.406597,291100.0,0.0,0,2009
5,2010-12-01,-0.553083,-0.619565,-0.540205,-0.608775,376800.0,0.0,0,2010
6,2011-12-01,-0.470012,-0.518941,-0.452961,-0.506338,189600.0,0.0,0,2011
7,2012-12-01,-0.618219,-0.641851,-0.602622,-0.641379,382700.0,0.0,0,2012
8,2013-12-01,-1.109750,-1.218942,-1.087466,-1.205572,619500.0,0.0,0,2013
9,2014-12-01,17.369110,18.862709,17.007756,17.698345,369400.0,0.0,0,2014


In [17]:
prices_filtered['ticker'] = ticker
prices_filtered['Growth -1'] = (prices_filtered['Close'] - prices_filtered['Close'].shift(periods=1)) /  prices_filtered['Close'].shift(periods=1)
prices_filtered['Growth +1'] = (prices_filtered['Close'].shift(periods=-1) - prices_filtered['Close']) /  prices_filtered['Close']
prices_filtered['Growth +5'] = (prices_filtered['Close'].shift(periods=-5) - prices_filtered['Close']) /  prices_filtered['Close']
prices_filtered['avgGrowth -10'] = prices_filtered['Growth -1'].rolling(min_periods=10, window=10).sum() / 10
prices_filtered['avgGrowth -5'] = prices_filtered['Growth -1'].rolling(min_periods=5, window=5).sum() / 5

# filter_years = pd.to_datetime(ticker_file['Date']).dt.year.tolist()
# histgrouped_filtered = prices_filtered[prices_filtered['year'].isin(filter_years)]
prices_filtered.reset_index(names=['longevity'], inplace=True)

In [18]:
prices_filtered.head(15)

,longevity,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,year,ticker,Growth -1,Growth +1,Growth +5,avgGrowth -10,avgGrowth -5
0,0,2005-12-01,-0.175528,-0.208950,-0.173211,-0.194557,2252900.0,0.0,0,2005,ITRN,NaN,-0.046737,2.129030,NaN,NaN
1,1,2006-12-01,-0.196678,-0.206167,-0.179426,-0.185464,912300.0,0.0,0,2006,ITRN,-0.046737,-0.206436,1.730115,NaN,NaN
2,2,2007-12-01,-0.136807,-0.148906,-0.132287,-0.147178,859900.0,0.0,0,2007,ITRN,-0.206436,0.393180,3.357855,NaN,NaN
3,3,2008-12-01,-0.216545,-0.250766,-0.200557,-0.205045,1223100.0,0.0,0,2008,ITRN,0.393180,0.982964,4.879552,NaN,NaN
4,4,2009-12-01,-0.411037,-0.433872,-0.401839,-0.406597,291100.0,0.0,0,2009,ITRN,0.982964,0.497245,-44.528021,NaN,NaN
5,5,2010-12-01,-0.553083,-0.619565,-0.540205,-0.608775,376800.0,0.0,0,2010,ITRN,0.497245,-0.168267,-26.932084,NaN,0.324043
6,6,2011-12-01,-0.470012,-0.518941,-0.452961,-0.506338,189600.0,0.0,0,2011,ITRN,-0.168267,0.266700,-46.421461,NaN,0.299737
7,7,2012-12-01,-0.618219,-0.641851,-0.602622,-0.641379,382700.0,0.0,0,2012,ITRN,0.266700,0.879657,-48.990909,NaN,0.394364
8,8,2013-12-01,-1.109750,-1.218942,-1.087466,-1.205572,619500.0,0.0,0,2013,ITRN,0.879657,-15.680455,-25.636830,NaN,0.491660
9,9,2014-12-01,17.369110,18.862709,17.007756,17.698345,369400.0,0.0,0,2014,ITRN,-15.680455,-0.108007,0.347281,NaN,-2.841024


In [ ]:
from market_growth_analysis.etl import web_scraping

In [ ]:
path = '../../data/intermediate_02/financial-sheets.csv'
ticker_file_all = pd.read_csv(path)

web_scraping.get_price_info('ITRN', ticker_file_all)

In [ ]:


# Read the csv from stagging
ticker_file = ticker_file_all[ticker_file_all['ticker']==ticker]

# Get the company data from yfinance
stock_info = yf.Ticker(ticker)
hist = stock_info.history(period="max", interval = "1mo")

# Transform data to add growth info
hist = hist.reset_index()
hist['Date'] = pd.to_datetime(hist['Date'])
hist['year'] = hist['Date'].dt.year
hist.dropna(inplace=True)

# Convert 'Date' column in 'hist' dataframe to string format
hist['Date'] = hist['Date'].astype(str)
# Extract the 'YYYY-MM' format from 'Date' column in 'ticker_file' dataframe
ticker_file['YearMonth'] = ticker_file['Date'].str.slice(0, 7)
# Filter 'hist' dataframe based on 'YYYY-MM' values from 'ticker_file'
filtered_hist = hist[hist['Date'].str.slice(0, 7).isin(ticker_file['YearMonth'])]



# Format to datetime
filtered_hist['Date'] = pd.to_datetime(filtered_hist['Date'])
hist['Date'] = pd.to_datetime(hist['Date'])
# Find the earliest date in the filtered_hist dataframe
earliest_date = filtered_hist['Date'].min()
# Filter 'hist' dataframe for rows with a date greater than the earliest date
filtered_hist_after_earliest = hist[hist['Date'].dt.year > earliest_date.year]
# Find the earliest date in the filtered_hist dataframe
earliest_date = filtered_hist['Date'].min()
# Filter 'hist' dataframe for rows with a date greater than the earliest date
filtered_hist_after_earliest = hist[hist['Date'].dt.year < earliest_date.year]
# Filter the resulting dataframe for rows with the same month as the earliest date
appended_hist = filtered_hist_after_earliest[filtered_hist_after_earliest['Date'].dt.month == earliest_date.month]
# Concatenate the filtered_hist dataframe with the appended_hist dataframe
prices_filtered = pd.concat([filtered_hist, appended_hist]).sort_values(by='year').reset_index(drop=True)



prices_filtered['ticker'] = ticker
prices_filtered['Growth -1'] = (prices_filtered['Close'] - prices_filtered['Close'].shift(periods=1)) /  prices_filtered['Close'].shift(periods=1)
prices_filtered['Growth +1'] = (prices_filtered['Close'].shift(periods=-1) - prices_filtered['Close']) /  prices_filtered['Close']
prices_filtered['Growth +5'] = (prices_filtered['Close'].shift(periods=-5) - prices_filtered['Close']) /  prices_filtered['Close']
prices_filtered['avgGrowth -10'] = prices_filtered['Growth -1'].rolling(min_periods=10, window=10).sum() / 10
prices_filtered['avgGrowth -5'] = prices_filtered['Growth -1'].rolling(min_periods=5, window=5).sum() / 5

# filter_years = pd.to_datetime(ticker_file['Date']).dt.year.tolist()
# histgrouped_filtered = prices_filtered[prices_filtered['year'].isin(filter_years)]
prices_filtered.reset_index(names=['longevity'], inplace=True)

In [ ]:
prices_filtered